# Necessary Imports

In [1]:
import numpy as np
import os
import glob
import tensorflow as tf
import argparse
import ecdl

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


# The arguments that we pass in

In [4]:
class myargs():
    def __init__(self):
        self.exp_id = 'efficientnet_round1'
        self.epochs = 200
        self.model = 'efficientnet'
        self.model_type = 'b3'
        self.image_dims = 256
        self.rotation_range = 60
        self.width_shift_range = 0.05
        self.height_shift_range = 0.05
        self.brightness_range = None
        self.zoom_range = 0.05
        self.horizontal_flip = True
        self.vertical_flip = True
        self.validation_split = 0.1
        self.batch_size = 8
        self.class_mode = 'categorical'
        
# {
#     "exp_id": "repeat_experiment_batch_10",
#     "learning_rate": 0.0001,
#     "epochs": 200,
#     "rotation_range": 60,
#     "width_shift_range": 0.05,
#     "height_shift_range": 0.05,
#     "brightness_range": null,
#     "zoom_range": 0.05,
#     "horizontal_flip": true,
#     "vertical_flip": true,
#     "validation_split": 0.1,
#     "model": "categorical_efficientnet",
#     "image_dims": 224,
#     "class_mode": "categorical",
#     "batch_size": 10
# }        

args = myargs()

# Data Generators

In [5]:
train_dir = '/workspaces/ecdl/elbow_data/SABINE_Training_Set'

base_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=args.rotation_range,
                                                                       width_shift_range=args.width_shift_range,
                                                                       height_shift_range=args.height_shift_range,
                                                                       brightness_range=args.brightness_range,
                                                                       zoom_range=args.zoom_range,
                                                                       horizontal_flip=args.horizontal_flip,
                                                                       vertical_flip=args.vertical_flip,
                                                                       rescale=1./255,
                                                                       validation_split = args.validation_split)



train_data_gen = base_image_generator.flow_from_directory(batch_size=args.batch_size,
                                                        target_size=(args.image_dims, args.image_dims),
                                                        directory=train_dir,
                                                        shuffle=True,
                                                        class_mode=args.class_mode,
                                                        subset='training')



val_data_gen = base_image_generator.flow_from_directory(batch_size=args.batch_size,
                                                        target_size=(args.image_dims, args.image_dims),
                                                        directory=train_dir,
                                                        shuffle=True,
                                                        class_mode=args.class_mode,
                                                        subset='validation')

Found 993 images belonging to 2 classes.
Found 109 images belonging to 2 classes.


In [6]:
sample = next(train_data_gen)

In [7]:
print(sample[0].shape)
print(sample[1].shape)
print(sample[0].shape[1:])

(8, 256, 256, 3)
(8, 2)
(256, 256, 3)


# Model

In [28]:
model = ecdl.models.tensorflow_efficientnet(model_name=args.model_type,
#                                             classes=2,
                                            weights='imagenet',
                                            include_top=True,
                                            input_shape=sample[0].shape[1:])                                      

model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']
             )
model.summary()

ValueError: When setting `include_top=True` and loading `imagenet` weights, `input_shape` should be (300, 300, 3).

In [7]:
model_base = ecdl.models.tensorflow_efficientnet(model_name=args.model_type,                                                 
                                                 weights='imagenet',
                                                 include_top=False,
                                                 input_shape=sample[0].shape[1:])                                      

# model_base.trainable = False

# Rebuild top
x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model_base.output)
# x = tf.keras.layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = tf.keras.layers.Dense(2, activation="softmax", name="pred")(x)

# Compile
model = tf.keras.Model(model_base.input, outputs, name="EfficientNet")
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)
model.summary()

Model: "EfficientNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 256, 256, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 257, 257, 3)  0           normalization[0][0]              
_______________________________________________________________________________________

In [10]:
from efficientnet.tfkeras import EfficientNetB3

efficient_net = EfficientNetB3(
        weights='imagenet',
        input_shape=(256, 256, 3),
        include_top=False,
#         pooling='max'
    )

# Basic
# x = tf.keras.layers.Dense(128, activation="relu", name="dense_1")(efficient_net.output)
# x = tf.keras.layers.Dense(128, activation="relu", name="dense_2")(x)
# outputs = tf.keras.layers.Dense(2, activation="softmax", name="pred")(x)

# Rebuild top
x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(efficient_net.output)
# x = tf.keras.layers.BatchNormalization()(x)
top_dropout_rate = 0.2
x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = tf.keras.layers.Dense(2, activation="softmax", name="pred")(x)

model = tf.keras.Model(efficient_net.input, outputs, name="EfficientNet")

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "EfficientNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 128, 128, 40) 1080        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 128, 128, 40) 160         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 128, 128, 40) 0           stem_bn[0][0]                    
_______________________________________________________________________________________

In [8]:
import tensorflow.keras.layers as Layers
model = tf.keras.Sequential()

model.add(Layers.Conv2D(200,kernel_size=(3,3),activation='relu',input_shape=(256,256,3)))
model.add(Layers.Conv2D(180,kernel_size=(3,3),activation='relu'))
model.add(Layers.MaxPool2D(5,5))
model.add(Layers.Conv2D(180,kernel_size=(3,3),activation='relu'))
model.add(Layers.Conv2D(140,kernel_size=(3,3),activation='relu'))
model.add(Layers.Conv2D(100,kernel_size=(3,3),activation='relu'))
model.add(Layers.Conv2D(50,kernel_size=(3,3),activation='relu'))
model.add(Layers.MaxPool2D(5,5))
model.add(Layers.Flatten())
model.add(Layers.Dense(180,activation='relu'))
model.add(Layers.Dense(100,activation='relu'))
model.add(Layers.Dense(50,activation='relu'))
model.add(Layers.Dropout(rate=0.5))
model.add(Layers.Dense(2,activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 200)     5600      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 252, 252, 180)     324180    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 180)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 180)       291780    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 46, 46, 140)       226940    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 44, 44, 100)       126100    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 42, 42, 50)        4

# Callbacks

In [9]:
exp_path = os.path.join('/workspaces/ecdl/experiments', args.exp_id)
checkpoint = tf.keras.callbacks.ModelCheckpoint(exp_path+'/weights.{epoch}-{val_accuracy:.4f}.hdf5', monitor='val_accuracy', verbose=1,save_best_only=True)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', verbose = 1, patience=50)
csv_logger = tf.keras.callbacks.CSVLogger(os.path.join(exp_path,'result.csv'))
delete_excess = ecdl.callbacks.delete_excess_callback(exp_folder=exp_path)
callbacks_list = [checkpoint, earlystopping, csv_logger, delete_excess]  

# Training

In [10]:
history = model.fit(
        train_data_gen,
        epochs = args.epochs,
        steps_per_epoch = len(train_data_gen),        
        validation_data = val_data_gen,
        verbose = 1,
        validation_steps = len(val_data_gen),
        callbacks=callbacks_list)

Epoch 1/200
125/125 [==============================] - ETA: 0s - loss: 0.6940 - accuracy: 0.5468
Epoch 00001: val_accuracy improved from -inf to 0.55046, saving model to /workspaces/ecdl/experiments/efficientnet_round1/weights.1-0.5505.hdf5
125/125 [==============================] - 28s 223ms/step - loss: 0.6940 - accuracy: 0.5468 - val_loss: 0.6913 - val_accuracy: 0.5505
Epoch 2/200
125/125 [==============================] - ETA: 0s - loss: 0.6905 - accuracy: 0.5498
Epoch 00002: val_accuracy did not improve from 0.55046
125/125 [==============================] - 27s 219ms/step - loss: 0.6905 - accuracy: 0.5498 - val_loss: 0.6873 - val_accuracy: 0.5505
Epoch 3/200
125/125 [==============================] - ETA: 0s - loss: 0.6917 - accuracy: 0.5468
Epoch 00003: val_accuracy did not improve from 0.55046
125/125 [==============================] - 28s 222ms/step - loss: 0.6917 - accuracy: 0.5468 - val_loss: 0.6891 - val_accuracy: 0.5505
Epoch 4/200
125/125 [==============================] 

KeyboardInterrupt: 

In [36]:
history = model.fit(
        train_data_gen,
        epochs = args.epochs,
        steps_per_epoch = len(train_data_gen),        
        validation_data = val_data_gen,
        verbose = 1,
        validation_steps = len(val_data_gen),
        callbacks=callbacks_list)

Epoch 1/10
89/89 [==============================] - ETA: 0s - loss: 1.0048 - accuracy: 0.5289
Epoch 00001: val_accuracy did not improve from 0.74886
89/89 [==============================] - 15s 168ms/step - loss: 1.0048 - accuracy: 0.5289 - val_loss: 445.8049 - val_accuracy: 0.4521
Epoch 2/10
89/89 [==============================] - ETA: 0s - loss: 0.8422 - accuracy: 0.5062
Epoch 00002: val_accuracy did not improve from 0.74886
89/89 [==============================] - 11s 121ms/step - loss: 0.8422 - accuracy: 0.5062 - val_loss: 28.8023 - val_accuracy: 0.4521
Epoch 3/10
89/89 [==============================] - ETA: 0s - loss: 0.7279 - accuracy: 0.5210
Epoch 00003: val_accuracy did not improve from 0.74886
89/89 [==============================] - 11s 124ms/step - loss: 0.7279 - accuracy: 0.5210 - val_loss: 1.3196 - val_accuracy: 0.4521
Epoch 4/10
89/89 [==============================] - ETA: 0s - loss: 0.7239 - accuracy: 0.5040
Epoch 00004: val_accuracy did not improve from 0.74886
89/89

KeyboardInterrupt: 

In [28]:
history = model.fit(
        train_data_gen,
        epochs = args.epochs,
        steps_per_epoch = len(train_data_gen),        
        validation_data = val_data_gen,
        verbose = 1,
        validation_steps = len(val_data_gen),
        callbacks=callbacks_list)

Epoch 1/10
89/89 [==============================] - ETA: 0s - loss: 1.0513 - accuracy: 0.6059
Epoch 00001: val_accuracy improved from 0.54795 to 0.63470, saving model to /workspaces/ecdl/experiments/efficientnet_round1/weights.1-0.6347.hdf5
89/89 [==============================] - 12s 133ms/step - loss: 1.0513 - accuracy: 0.6059 - val_loss: 0.6536 - val_accuracy: 0.6347
Epoch 2/10
89/89 [==============================] - ETA: 0s - loss: 0.5819 - accuracy: 0.7305
Epoch 00002: val_accuracy did not improve from 0.63470
89/89 [==============================] - 11s 120ms/step - loss: 0.5819 - accuracy: 0.7305 - val_loss: 0.6673 - val_accuracy: 0.5982
Epoch 3/10
89/89 [==============================] - ETA: 0s - loss: 0.5006 - accuracy: 0.7667
Epoch 00003: val_accuracy improved from 0.63470 to 0.65753, saving model to /workspaces/ecdl/experiments/efficientnet_round1/weights.3-0.6575.hdf5
89/89 [==============================] - 11s 125ms/step - loss: 0.5006 - accuracy: 0.7667 - val_loss: 0.8

In [9]:
history = model.fit(
        train_data_gen,
        epochs = 1000,
        steps_per_epoch = len(train_data_gen),        
        validation_data = val_data_gen,
        verbose = 1,
        validation_steps = len(val_data_gen),
        callbacks=callbacks_list)

Epoch 1/1000
89/89 [==============================] - ETA: 0s - loss: 0.7437 - accuracy: 0.5900
Epoch 00001: val_accuracy improved from -inf to 0.61644, saving model to /workspaces/ecdl/experiments/efficientnet_round1/weights.1-0.6164.hdf5
89/89 [==============================] - 21s 238ms/step - loss: 0.7437 - accuracy: 0.5900 - val_loss: 0.6549 - val_accuracy: 0.6164
Epoch 2/1000
89/89 [==============================] - ETA: 0s - loss: 0.6048 - accuracy: 0.6852
Epoch 00002: val_accuracy improved from 0.61644 to 0.68037, saving model to /workspaces/ecdl/experiments/efficientnet_round1/weights.2-0.6804.hdf5
89/89 [==============================] - 19s 214ms/step - loss: 0.6048 - accuracy: 0.6852 - val_loss: 0.5395 - val_accuracy: 0.6804
Epoch 3/1000
89/89 [==============================] - ETA: 0s - loss: 0.5214 - accuracy: 0.7463
Epoch 00003: val_accuracy improved from 0.68037 to 0.72146, saving model to /workspaces/ecdl/experiments/efficientnet_round1/weights.3-0.7215.hdf5
89/89 [===

Epoch 28/1000
89/89 [==============================] - ETA: 0s - loss: 0.0371 - accuracy: 0.9864
Epoch 00028: val_accuracy did not improve from 0.78995
89/89 [==============================] - 19s 212ms/step - loss: 0.0371 - accuracy: 0.9864 - val_loss: 1.0880 - val_accuracy: 0.7306
Epoch 29/1000
89/89 [==============================] - ETA: 0s - loss: 0.0417 - accuracy: 0.9887
Epoch 00029: val_accuracy did not improve from 0.78995
89/89 [==============================] - 20s 220ms/step - loss: 0.0417 - accuracy: 0.9887 - val_loss: 0.9293 - val_accuracy: 0.7580
Epoch 30/1000
89/89 [==============================] - ETA: 0s - loss: 0.0121 - accuracy: 0.9966
Epoch 00030: val_accuracy did not improve from 0.78995
89/89 [==============================] - 20s 220ms/step - loss: 0.0121 - accuracy: 0.9966 - val_loss: 1.2320 - val_accuracy: 0.7580
Epoch 31/1000
89/89 [==============================] - ETA: 0s - loss: 0.0024 - accuracy: 1.0000
Epoch 00031: val_accuracy improved from 0.78995 to 

Epoch 57/1000
89/89 [==============================] - ETA: 0s - loss: 0.0323 - accuracy: 0.9921
Epoch 00057: val_accuracy did not improve from 0.79909
89/89 [==============================] - 19s 217ms/step - loss: 0.0323 - accuracy: 0.9921 - val_loss: 1.5587 - val_accuracy: 0.7032
Epoch 58/1000
89/89 [==============================] - ETA: 0s - loss: 0.0459 - accuracy: 0.9853
Epoch 00058: val_accuracy did not improve from 0.79909
89/89 [==============================] - 19s 217ms/step - loss: 0.0459 - accuracy: 0.9853 - val_loss: 1.0214 - val_accuracy: 0.7443
Epoch 59/1000
89/89 [==============================] - ETA: 0s - loss: 0.0434 - accuracy: 0.9875
Epoch 00059: val_accuracy did not improve from 0.79909
89/89 [==============================] - 19s 215ms/step - loss: 0.0434 - accuracy: 0.9875 - val_loss: 1.0478 - val_accuracy: 0.7306
Epoch 60/1000
89/89 [==============================] - ETA: 0s - loss: 0.0158 - accuracy: 0.9966
Epoch 00060: val_accuracy did not improve from 0.79